In [1]:
from lxml import etree
from bs4 import BeautifulSoup
from parsel import Selector

import pandas as pd
import numpy as np

import regex as re

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
pio.renderers.default = "vscode"

In [2]:
# with open('john-hole.html') as fp:
#     soup = BeautifulSoup(fp, 'html.parser')
with open('john-hole.html') as f:
    html = f.read()


In [3]:
html = html.replace('<b>', '')
html = html.replace('</b>', '')
html = html.replace('\n', '')

In [4]:
selector = Selector(html)

In [5]:
# selector.xpath('//div/span[@class="bubble"]/text()').getall()
message_objs = {
    'texts' : [],
    'image' : [],
    'sender_number' : [],
    'reactions' : []
}
dates = []
for message in selector.xpath('//div[@class="message"]'):
    dates.append(re.sub(r'\(.*\)', '', message.xpath('./div/p/span[@class="timestamp"]/text()').get()))
    message_objs['sender_number'].append(message.xpath('./div/p/span[@class="sender"]/text()').get())
    message_objs['texts'].append(message.xpath('./div/div/span[@class="bubble"]/text()').get())
    message_objs['image'].append(message.xpath('./div/div/img/@src').get())
    reactions = message.xpath('./div/div[@class="reactions"]').xpath('./div[@class="reaction"]/span/text()').getall()
    message_objs['reactions'].append([reaction.replace('\t', '') for reaction in reactions])

In [6]:
message_objs['dates'] = dates
df = pd.DataFrame(message_objs)

In [7]:
df

,texts,image,sender_number,reactions,dates
0,Bro surrrrreeee,None,Me,[],"Jan 01, 2025 12:43:16 AM"
1,You fucking dick,None,+16138588921,[],"Jan 01, 2025 12:46:22 AM"
2,Ben Riley darcy ryan please send me a custom c...,None,+15198782573,[Laughed by +14169857677],"Jan 01, 2025 12:47:41 AM"
3,^^,None,+16133559273,[],"Jan 01, 2025 12:48:44 AM"
4,What about hack,None,+14169857677,[],"Jan 01, 2025 12:53:11 AM"
...,...,...,...,...,...
15270,"Jack, we could be like the white stripes, I’ll...",None,+16138588921,[Laughed by itsjaxonmusic@gmail.com],"Jan 31, 2025 10:53:01 AM"
15271,LLOOOOOOOOOOL,None,itsjaxonmusic@gmail.com,[],"Jan 31, 2025 10:53:41 AM"
15272,apmed for hole stats today🤓,None,+14163054266,[],"Jan 31, 2025 10:57:23 AM"
15273,None,None,+14163054266,[],"Jan 31, 2025 10:57:40 AM"


In [8]:
# Dropping duplicates
df = df.loc[df[['texts', 'sender_number', 'dates']].drop_duplicates().index]

In [9]:
df.dates = df.dates.str.strip()

In [10]:
df.set_index('dates', inplace=True)

In [11]:
df.index = pd.to_datetime(df.index, format="%b %d, %Y %I:%M:%S %p")

In [12]:
df

,texts,image,sender_number,reactions
dates,,,,
2025-01-01 00:43:16,Bro surrrrreeee,None,Me,[]
2025-01-01 00:46:22,You fucking dick,None,+16138588921,[]
2025-01-01 00:47:41,Ben Riley darcy ryan please send me a custom c...,None,+15198782573,[Laughed by +14169857677]
2025-01-01 00:48:44,^^,None,+16133559273,[]
2025-01-01 00:53:11,What about hack,None,+14169857677,[]
...,...,...,...,...
2025-01-31 10:53:01,"Jack, we could be like the white stripes, I’ll...",None,+16138588921,[Laughed by itsjaxonmusic@gmail.com]
2025-01-31 10:53:41,LLOOOOOOOOOOL,None,itsjaxonmusic@gmail.com,[]
2025-01-31 10:57:23,apmed for hole stats today🤓,None,+14163054266,[]


In [13]:
df.sender_number.unique()

array(['Me', '+16138588921', '+15198782573', '+16133559273',
       '+14169857677', '+12368865666', '+16138044084', '+16472994537',
       'gregorynip@icloud.com', '+16133160960',
       'hashtagyoloswagbeef@gmail.com', '+16476226650', '+14372341484',
       '+19024973878', '+16048685587', '+16136202840', '+14168368964',
       '+14168347529', '+12899330628', '+14167995612', '+16474591137',
       'josh.maycock@yahoo.com', '+13432045724',
       'melaniesmyth13@gmail.com', '+16134004076', '+16138093940',
       '+16475754548', '+14163054266', '+16477019241',
       'stephazel108@gmail.com', '+16135581208', '+16479917601',
       'itsjaxonmusic@gmail.com', '+16475394413', '+16139797154',
       'edenkettleson@gmail.com', '+15197192478',
       'ernestormz99@gmail.com', 'ryanmatson18@gmail.com', '+18103048098',
       '+16138594804'], dtype=object)

In [14]:
senders = {
    '+16138044084' : 'Ryan',
    'ryan.matson@icloud.com' : 'Ryan',
    '+16472994537' : 'Greg',
    '+16476226650' : 'Ali',
    '+15198782573' : 'John',
    '+14168347529' : 'Tommy',
    '+14383514096' : 'Jake',
    '+16138588921' : 'Ben',
    'ben_maycock@yahoo.com' : 'Ben',
    '+16048685587' : 'Michaela',
    '+19024973878' : 'Ryan 2',
    'ryan.keays@gmail.com' : 'Ryan 2',
    'darcydrums@yahoo.ca' : 'Darcy',
    '+447918720754' : 'Darcy',
    '+16136202840' : 'Darcy',
    'Me' : 'Henry',
    '+16474679692' : 'Pias',
    '+16133160960' : 'Jack',
    'itsjaxonmusic@gmail.com' : 'Jack',
    '+436764425651' : 'Miklos',
    '+14167055056' : 'Ernesto',
    '+14168368964' : 'Rayhan',
    '+12263763043' : 'Maggie',
    '+14169857677' : 'Riley',
    '+14372341484' : 'M****',
    '+14372555533' : 'Colin',
    'ernestormz99@gmail.com' : 'Ernesto',
    '+16133559273' : 'Igor',
    '+16475754548' : 'Nicole',
    '+16137628867' : 'Mel',
    'gregorynip@icloud.com' : 'Greg', 
    'edenkettleson@gmail.com' : 'Eden',
    '+12368865666' : 'Eden',
    '+16474591137' : 'Krystiana',
    '+14163054266' : 'Austin',
    'ryanmatson18@gmail.com' : 'Ryan',
    '+16134004076' : 'Nick'
}

In [15]:
df['reactions_recieved'] = df['reactions'].apply(lambda x : len(x))

In [16]:
df['reactions_recieved']

dates
2025-01-01 00:43:16    0
2025-01-01 00:46:22    0
2025-01-01 00:47:41    1
2025-01-01 00:48:44    0
2025-01-01 00:53:11    0
                      ..
2025-01-31 10:53:01    1
2025-01-31 10:53:41    0
2025-01-31 10:57:23    0
2025-01-31 10:57:40    0
2025-01-31 10:59:04    0
Name: reactions_recieved, Length: 13282, dtype: int64

In [17]:
df['sender'] = df['sender_number'].replace(senders)

In [18]:
text_count = df.groupby('sender').count().sort_values(by='texts', ascending=False)

In [19]:
text_count['reactions_recieved'] = df.groupby('sender').sum()['reactions_recieved']

In [20]:
text_count['messages'] = text_count['texts'] + text_count['image']
text_count = text_count.sort_values(by='messages', ascending=False)

In [21]:
# Tommy reacted but did not send any messages

# tommy_dict = [{col: 0 for col in text_count.columns}]
# tommy_dict[0]['sender'] = 'Tommy'
# tommy_df = pd.DataFrame(tommy_dict)
# tommy_df = tommy_df.set_index('sender')
# text_count = pd.concat([text_count, tommy_df])

In [22]:
text_count[['texts', 'image', 'messages']]

,texts,image,messages
sender,,,
Austin,1547,128,1675
Krystiana,1549,120,1669
Ryan,1412,239,1651
Ben,1215,61,1276
Igor,1097,87,1184
Riley,988,131,1119
John,604,363,967
Nick,827,91,918
Jack,605,86,691


In [23]:
text_count['reactions_sent'] = pd.Series()
text_count['reactions_sent'] = text_count['reactions_sent'].fillna(0)

/var/folders/zm/kglhp53x58752bpgh6114_f40000gn/T/ipykernel_94138/1290227952.py:2: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [24]:
text_count['hahas_sent'] = pd.Series()
text_count['hahas_sent'] = text_count['hahas_sent'].fillna(0)

/var/folders/zm/kglhp53x58752bpgh6114_f40000gn/T/ipykernel_94138/2859609646.py:2: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [25]:
def reactions_sent(x):
  for reaction in x:
    for sender in df.sender_number.unique():
      if sender in reaction:
        try:
          text_count.loc[senders.get(sender, sender), 'reactions_sent'] += 1
        except KeyError:
          pass

In [26]:
def hahas_sent(x):
  for reaction in x:
    for sender in df.sender_number.unique():
      if (sender in reaction) and ("Laughed" in reaction):
        try:
          text_count.loc[senders.get(sender, sender), 'hahas_sent'] += 1
        except KeyError:
          pass

In [27]:
df['reactions'].apply(reactions_sent)
df['reactions'].apply(hahas_sent)
text_count[['texts', 'image', 'reactions_recieved', 'reactions_sent', 'hahas_sent', 'messages']]

,texts,image,reactions_recieved,reactions_sent,hahas_sent,messages
sender,,,,,,
Austin,1547,128,540,629,341,1675
Krystiana,1549,120,330,280,205,1669
Ryan,1412,239,1017,568,450,1651
Ben,1215,61,585,644,515,1276
Igor,1097,87,565,363,279,1184
Riley,988,131,449,733,619,1119
John,604,363,649,194,131,967
Nick,827,91,408,751,603,918
Jack,605,86,433,730,645,691


In [28]:
px.bar(text_count.loc[text_count['messages'] > 5]['messages'], labels={'value' : '# of Messages', 'variable' : 'Message Type', 'sender' : 'Sender'})

In [29]:
px.bar(text_count.loc[text_count['messages'] > 5][['texts', 'image']], labels={'value' : '# of Messages', 'variable' : 'Message Type', 'sender' : 'Sender'}, title="Messages Sent")

/Users/henrybrown/opt/anaconda3/envs/data_analysis/lib/python3.9/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [47]:
list(df['texts'])

['Bro surrrrreeee',
 'You fucking dick',
 'Ben Riley darcy ryan please send me a custom cameo video right now',
 '^^',
 'What about hack',
 'Hack',
 'Jack',
 'Jack moss rizzing all the huzz',
 None,
 'Who’s the guy infront of Luigi mangione',
 'Graeme',
 None,
 'Dart circle',
 None,
 'LOOOL',
 'LMAOOO',
 'HOW DID YOU DO THAT SO FAST',
 None,
 'John hole new years!!!!',
 'J hole',
 None,
 None,
 'Riley darcy nye',
 'Onece in a lifetime',
 'Happy new year :)!',
 'Please send to eden',
 'Incredible',
 'Man jack is fucking hitting that',
 None,
 'Salmen 😍',
 'Dear hole,',
 'Please state your 2025 Ins and 2025 outs',
 'I can’t put down the mowater I can’t put down the mowater',
 None,
 'Stop firing at Ukraine dawg',
 'New jack pic that doesn’t look like jack dropped',
 'LOOOOL',
 None,
 'InMowaterJosh (Cabernet Sauvignon)Josh (Maycock)Shawshank redemptionCRT Television static OutA&W Iced teaHot tubs that are too smallGreening out at new years',
 'LOOOL',
 'Dude Josh coming over to Evan’s ho

In [30]:
df['texts'].fillna('')

dates
2025-01-01 00:43:16                                      Bro surrrrreeee
2025-01-01 00:46:22                                     You fucking dick
2025-01-01 00:47:41    Ben Riley darcy ryan please send me a custom c...
2025-01-01 00:48:44                                                   ^^
2025-01-01 00:53:11                                      What about hack
                                             ...                        
2025-01-31 10:53:01    Jack, we could be like the white stripes, I’ll...
2025-01-31 10:53:41                                        LLOOOOOOOOOOL
2025-01-31 10:57:23                          apmed for hole stats today🤓
2025-01-31 10:57:40                                                     
2025-01-31 10:59:04                                We have stats coming?
Name: texts, Length: 13282, dtype: object

In [31]:
df['word_count'] = df['texts'].fillna('').apply(lambda x : len(x.split(' ')))

In [32]:
word_count = df.groupby('sender').sum().sort_values(by='word_count', ascending=False)

In [50]:
text_count['word_count'] = word_count.word_count

In [33]:
fig = go.Figure(data=[go.Bar(
            x=word_count.index, y=word_count['word_count'],
            text=word_count['word_count'],
            textposition='auto',
        )])

fig.update_layout(
    title="Word Count",
    xaxis_title="Sender",
    yaxis_title="Words",
)

fig.show()

In [34]:
fig = go.Figure(data=[go.Bar(
            x=text_count['hahas_sent'].sort_values(ascending=False).index, y=text_count['hahas_sent'].sort_values(ascending=False),
            text=text_count['hahas_sent'].sort_values(ascending=False),
            textposition='auto',
        )])

fig.update_layout(
    title="Hahas Sent",
    xaxis_title="Sender",
    yaxis_title="Haha Count",
)

fig.show()

In [44]:
(text_count.loc[text_count['messages'] > 5]['reactions_recieved'] / text_count.loc[text_count['messages'] > 5]['messages']).sort_values(ascending=False)

sender
Ali                              1.761905
Greg                             0.958678
John                             0.671148
Henry                            0.662921
Darcy                            0.654762
Jack                             0.626628
melaniesmyth13@gmail.com         0.616541
Ryan                             0.615990
Rayhan                           0.600000
hashtagyoloswagbeef@gmail.com    0.569288
Igor                             0.477196
Eden                             0.476923
josh.maycock@yahoo.com           0.475000
Ben                              0.458464
Nick                             0.444444
Riley                            0.401251
+14167995612                     0.328467
Austin                           0.322388
+12899330628                     0.303571
Ernesto                          0.259259
Krystiana                        0.197723
Nicole                           0.166667
+16475394413                     0.090909
dtype: float64

In [61]:
text_count[["texts", "image", "reactions_recieved", "messages", "reactions_sent", "hahas_sent", "ratio", "word_count"]].to_json('data.json', orient='index')

## This is where all of the formatting ends and the file has been outputted

In [36]:
df.loc[df.sender == 'Ali']

,texts,image,sender_number,reactions,reactions_recieved,sender,word_count
dates,,,,,,,
2025-01-01 22:28:30,None,None,+16476226650,[],0,Ali,1
2025-01-01 22:49:30,None,None,+16476226650,[],0,Ali,1
2025-01-03 12:49:48,I love you John! Happy bday💋,output-html-jan31/attachments/341/d2e376fd-bad...,+16476226650,[Loved by Me],1,Ali,6
2025-01-06 14:44:03,None,output-html-jan31/attachments/341/2a219600-7eb...,+16476226650,"[Laughed by +14169857677, Laughed by Me]",2,Ali,1
2025-01-12 00:26:57,Maya!!! Maya the great,None,+16476226650,[],0,Ali,4
2025-01-14 21:28:40,System attack,None,+16476226650,[],0,Ali,2
2025-01-17 00:33:29,None,output-html-jan31/attachments/341/304568fd-c98...,+16476226650,"[Laughed by gregorynip@icloud.com, Laughed by ...",2,Ali,1
2025-01-23 10:57:48,Oh man… there’s no way it’s coming back,None,+16476226650,[Laughed by edenkettleson@gmail.com],1,Ali,8
2025-01-23 16:52:30,thank you,None,+16476226650,[],0,Ali,2


In [37]:
df.loc[df.reactions_recieved == df.reactions_recieved.max()].texts.to_string()

'dates\n2025-01-22 21:19:20    RICE 🍚 EGG 🥚 ONION 🧅 RICE 🍚 EGG 🥚 ONION 🧅RICE ...'

In [38]:
text_count['ratio'] = (text_count['reactions_recieved'] / text_count['messages']).sort_values(ascending=False)

In [39]:
text_count.sort_values(ascending=False, by='ratio')[['messages', 'reactions_recieved']]

,messages,reactions_recieved
sender,,
Ali,42,74
+13432045724,4,5
+16135581208,5,6
M****,1,1
Tommy,1,1
stephazel108@gmail.com,1,1
Greg,121,116
John,967,649
Henry,178,118


In [40]:
px.bar(text_count['ratio'].sort_values(ascending=False), title="Text-to-Reactions Recieved Ratio")

In [41]:
df.loc[df.sender == "Ali"].sort_values("reactions_recieved", ascending=False)[['texts', 'reactions_recieved']].values

array([['Loving john is a treacherous journey full of emotions', 6],
       ['I’m so glad I’m not Johns friend. I would hate to be betrayed like this',
        6],
       [None, 5],
       ['Oh. Sorry. I don’t see you like that', 4],
       ['Behind the scenes of john holing', 4],
       ['I am currently working at a company called Fable. They help make digital products accessible for people with disabilities',
        4],
       ['This looks exactly like Luigi', 3],
       ['I resent that', 3],
       ['This chat is actually so freaking funny. I was up late last night and just cackling at some shit',
        3],
       ['Infancy. Its hard. You can tell at times he really wants to be able to speak .',
        3],
       ['Elevated Darcy’s crate', 3],
       ['Omg… Ryan stop!!! I’m a girl and I’m taken', 3],
       ['Happy birthday little Henry!!!!!!', 3],
       [None, 3],
       ['I feel crazy too much', 2],
       ['This is Ryan’s type', 2],
       ['Good afternoon', 2],
       [None

In [42]:
df.loc[df.reactions_recieved.idxmax()]['reactions']

[' from +16138588921',
 ' from +15198782573',
 ' from +16138044084',
 ' from +16138044084',
 ' from +16138044084',
 ' from +16138044084',
 ' from +16138044084',
 ' from +14169857677',
 ' from +15198782573',
 ' from +14169857677',
 ' from +16138588921',
 ' from +16138588921',
 ' from +16138588921',
 ' from +16138588921']

In [43]:
px.line(
  df[(df.sender == 'Ryan 1') | (df.sender == 'Greg')], x=df[(df.sender == 'Ryan 1') | (df.sender == 'Greg')].index, y='word_count', color='sender',
  labels={'word_count' : 'Word Count', 'x' : 'Date'}
)

/Users/henrybrown/opt/anaconda3/envs/data_analysis/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

